#### Framework de Big Data

#### Conteúdo - Bases e Notebook da aula

Base disponivel no caminho:  

https://github.com/FIAP/Pos_Tech_DTAT/tree/Framework-de-Big-Data

E para correto funcionamento foi necessario a criação de um roteiro de instalação e configuração: 

https://github.com/RicardViana/fiap-Big-Data/blob/main/PySpark%20para%20VS%20Code%20e%20Anaconda.pdf

Como não é possivel usar o Spark para ler diretamnte o arquivo no Github, usar o modelo abaixo  

```python

url = "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/Framework-de-Big-Data/Aula%201/banklist.csv"
local_path = "banklist.csv"
urllib.request.urlretrieve(url, local_path)

df = spark.read.csv(local_path, sep=",", header=True, inferSchema=True)

```

Caso demore muito para executar, recomendo reiniciar o Kernel

#### Importação de pacotes, bibliotecas e funções (def)

In [76]:
# Importar biblioteca completa
import pandas as pd 
import findspark
import urllib.request

# Importar função especifica de um módulo
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Import spark libraries
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType

#from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql.functions import *

from pyspark.sql import functions as F
from functools import reduce

In [77]:
# Como não queremos configurar as variaveis de ambiente, precisamos usar o findspark.init
findspark.init()

#### Aula 1 - Conhecendo o Spark

In [78]:
"""

1. spark = SparkSession.builder.master("local[*]").getOrCreate()
Esta é a linha que inicializa o Spark. Ela é o ponto de entrada para qualquer funcionalidade do Spark.

SparkSession: É o principal ponto de entrada para a programação com DataFrames e Datasets no Spark.

.builder: É um padrão de projeto (builder pattern) usado para construir o objeto SparkSession com diferentes configurações.

.master("local[*]"): Esta é uma das configurações mais importantes. Ela define como e onde o Spark irá executar suas tarefas.

"local[*]" instrui o Spark a ser executado em modo local, utilizando todos os núcleos de processamento (cores) disponíveis na sua máquina. 
Se você quisesse usar apenas 2 núcleos, por exemplo, usaria "local[2]". Este modo é ideal para desenvolvimento, testes e aprendizado em uma única máquina.

.getOrCreate(): Este método cria uma nova SparkSession se uma não existir. Se já houver uma sessão ativa com as mesmas configurações, ele simplesmente a retorna. 
Isso evita a criação de múltiplas sessões desnecessariamente.

Em resumo: esta linha cria e configura uma sessão do Spark para rodar localmente na sua máquina, usando todos os recursos de processamento disponíveis, e armazena essa sessão na variável spark.

2. df = spark.read.csv(link, sep= ",", inferSchema = True, header = True)
Esta é a linha que efetivamente lê os dados e os transforma em um DataFrame, que é a principal estrutura de dados do Spark.

spark.read: É o objeto usado para ler dados de fontes externas (arquivos, bancos de dados, etc.).

.csv(link, ...): Especifica que o formato do arquivo a ser lido é CSV. O primeiro argumento, link, é o caminho para o arquivo.

sep= ",": Este parâmetro (separador) informa ao Spark que as colunas no arquivo CSV são delimitadas por uma vírgula. É o padrão para arquivos CSV, mas é uma boa prática especificá-lo.

inferSchema = True: Esta é uma opção muito útil. Ela instrui o Spark a analisar uma amostra dos dados para inferir (deduzir) automaticamente o tipo de dado de cada coluna. 
Por exemplo, ele tentará identificar se uma coluna contém números inteiros (IntegerType), números decimais (DoubleType) ou texto (StringType). Sem isso, todas as colunas seriam tratadas como texto (String).

header = True: Esta opção informa ao Spark que a primeira linha do arquivo CSV contém os nomes das colunas (cabeçalho) e que essa linha não deve ser tratada como dados. 
O Spark usará esses nomes para as colunas do DataFrame.

"""

spark = SparkSession.builder.master("local[*]").getOrCreate()

# Baixar o arquivo pois não conseguimos usar o Spark para ler diretamnte o arquivo no Github
url = "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/Framework-de-Big-Data/Aula%201/banklist.csv"
local_path = "banklist.csv"
urllib.request.urlretrieve(url, local_path)

# Ler com Spark
df = spark.read.csv(local_path, sep=",", header=True, inferSchema=True)

print(f"df.count: {df.count()}")
print(f"df.col ct: {len(df.columns)}")
print(f"df.columns: {df.columns}")

df.count: 561
df.col ct: 6
df.columns: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


In [79]:
# Criar uma tabela temporaria
df.createOrReplaceTempView("banklist")

# Fazer o select 
df_check = spark.sql(''' select `Bank Name`, `City`,`Closing Date` from banklist ''')
df_check.show(4, truncate=False) # --> Retornar apenas os 4 primeiros dados e não truncar textos longos

+--------------------------------+-------------+------------+
|Bank Name                       |City         |Closing Date|
+--------------------------------+-------------+------------+
|The First State Bank            |Barboursville|3-Apr-20    |
|Ericson State Bank              |Ericson      |14-Feb-20   |
|City National Bank of New Jersey|Newark       |1-Nov-19    |
|Resolute Bank                   |Maumee       |25-Oct-19   |
+--------------------------------+-------------+------------+
only showing top 4 rows


In [80]:
# Executar a estatistica descritiva 
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                NULL|   NULL|NULL|31685.68449197861|                 NULL|        NULL|
| stddev|                NULL|   NULL|NULL|16446.65659309965|                 NULL|        NULL|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [81]:
# Executar a estatistica descritiva 
df.describe('City', 'ST').show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   NULL|NULL|
| stddev|   NULL|NULL|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



In [82]:
# Ver informações do data frame 
print(f"Total de linhas: {df.count()}")
print(f"Total de colunas: {len(df.columns)}")
print(f"Colunas: {df.columns}")
print(f"Tipo de dados: {df.dtypes}")
print(f"Schema {df.schema}")


Total de linhas: 561
Total de colunas: 6
Colunas: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Tipo de dados: [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]
Schema StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])


In [83]:
# Ver o Schema dos dados 
df.printSchema()

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)



In [84]:
# Remover duplicadas 
df = df.dropDuplicates()
print(f"df.count: {df.count()}")
print(f"df.columns: {df.columns}")

df.count: 561
df.columns: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


In [85]:
# Selecionar colunas especificas do Data Set
df2 = df.select(*['Bank Name', 'City'])
df2.show(truncate=False)

+---------------------------------+----------------+
|Bank Name                        |City            |
+---------------------------------+----------------+
|First Bank of Idaho              |Ketchum         |
|Amcore Bank, National Association|Rockford        |
|Venture Bank                     |Lacey           |
|First State Bank of Altus        |Altus           |
|Valley Capital Bank, N.A.        |Mesa            |
|Michigan Heritage Bank           |Farmington Hills|
|Columbia Savings Bank            |Cincinnati      |
|Fidelity Bank                    |Dearborn        |
|The Park Avenue Bank             |Valdosta        |
|Western Commercial Bank          |Woodland Hills  |
|Syringa Bank                     |Boise           |
|Republic Federal Bank, N.A.      |Miami           |
|Westside Community Bank          |University Place|
|First United Bank                |Crete           |
|HarVest Bank of Maryland         |Gaithersburg    |
|BankEast                         |Knoxville  

In [86]:
# Selecionar colunas exceto o que tiver na lista 
col_l = list(set(df.columns) - {'ST', 'CERT'})
df2 = df.select(*col_l)
df2.show(truncate=False)

+------------+---------------------------------+----------------+----------------------------------------------------+
|Closing Date|Bank Name                        |City            |Acquiring Institution                               |
+------------+---------------------------------+----------------+----------------------------------------------------+
|24-Apr-09   |First Bank of Idaho              |Ketchum         |U.S. Bank, N.A.                                     |
|23-Apr-10   |Amcore Bank, National Association|Rockford        |Harris N.A.                                         |
|11-Sep-09   |Venture Bank                     |Lacey           |First-Citizens Bank & Trust Company                 |
|31-Jul-09   |First State Bank of Altus        |Altus           |Herring Bank                                        |
|11-Dec-09   |Valley Capital Bank, N.A.        |Mesa            |Enterprise Bank & Trust                             |
|24-Apr-09   |Michigan Heritage Bank           |

In [87]:
# Renomear as colunas 
df2 = df \
  .withColumnRenamed('Bank Name'            , 'bank_name') \
  .withColumnRenamed('Acquiring Institution', 'acq_institution') \
  .withColumnRenamed('Closing Date'         , 'closing_date') \
  .withColumnRenamed('ST'                   , 'state') \
  .withColumnRenamed('CERT'                 , 'cert') 

df2.show()

+--------------------+----------------+-----+-----+--------------------+------------+
|           bank_name|            City|state| cert|     acq_institution|closing_date|
+--------------------+----------------+-----+-----+--------------------+------------+
| First Bank of Idaho|         Ketchum|   ID|34396|     U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|   IL| 3735|         Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey|   WA|22868|First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|   OK| 9873|        Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa|   AZ|58399|Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|   MI|34369|      Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati|   OH|32284|United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn|   MI|33883|The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   GA|19797|  B

In [88]:
# Adicionar uma nova coluna 
df2 = df.withColumn('State', col('ST'))
df2.show(5)

+--------------------+--------+---+-----+---------------------+------------+-----+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|State|
+--------------------+--------+---+-----+---------------------+------------+-----+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|   ID|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|   IL|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|   WA|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|   OK|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|   AZ|
+--------------------+--------+---+-----+---------------------+------------+-----+
only showing top 5 rows


In [89]:
# Adicionar uma nova coluna
df2 = df.withColumn('country', lit('US'))
df2.show(5)

+--------------------+--------+---+-----+---------------------+------------+-------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+--------+---+-----+---------------------+------------+-------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|     US|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|     US|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|     US|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|     US|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|     US|
+--------------------+--------+---+-----+---------------------+------------+-------+
only showing top 5 rows


In [90]:
# Apagar coluna 
df2 = df.drop('CERT')
df2.show(5)

+--------------------+--------+---+---------------------+------------+
|           Bank Name|    City| ST|Acquiring Institution|Closing Date|
+--------------------+--------+---+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| WA| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus| OK|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| AZ| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---+---------------------+------------+
only showing top 5 rows


In [91]:
# Apagar varias colunas 
df2 = df.drop(*["CERT","ST"])
df2.show(5)

+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---------------------+------------+
only showing top 5 rows


In [92]:
# Apagar varias colunas com reduce 
df2 = reduce(DataFrame.drop, ["CERT","ST"], df)
df2.show(5)

+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---------------------+------------+
only showing top 5 rows


In [93]:
# Realizar filtros 
df2 = df.where(df['ST'] == 'NE')
df3 = df.where(df['CERT'].between('1000','2000'))
df4 = df.where(df['ST'].isin('NE','IL'))

print('df.count  :', df.count())
print('df2.count :', df2.count())
print('df3.count :', df3.count())
print('df4.count :', df4.count())


df.count  : 561
df2.count : 4
df3.count : 9
df4.count : 73


In [94]:
# Realizar filtros --> AND
df2 = df.where((df['ST'] == 'NE') & (df['City'] == 'Ericson'))
df2.show(3)

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



In [95]:
# Alterar os dados --> Replace
df.show(5)

print('Replace 7 in the above dataframe with 17 at all instances')
df.na.replace(7,17).show(5) # --> Dessa forma, substitui em todas as colunas o que for 7 por 17

+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---+-----+---------------------+------------+
only showing top 5 rows
Replace 7 in the above dataframe with 17 at all instances
+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+--------

#### Aula 2 - Operações Básicas no Spark

In [96]:
"""

O trecho de código sc = SparkContext.getOrCreate() é uma instrução fundamental em aplicações Apache Spark, especialmente em versões mais antigas da tecnologia. 
Ele é responsável por inicializar o ponto de entrada principal para a funcionalidade do Spark, o SparkContext. Vamos detalhar o que cada parte significa.

O que é o SparkContext?
O SparkContext (geralmente abreviado como sc) é o coração de uma aplicação Spark. Ele representa a conexão com um cluster Spark e é usado para coordenar os processos que serão
executados nos nós de trabalho (workers) desse cluster. Pense nele como o "maestro" da sua orquestra de processamento de dados distribuídos.

As principais funções de um SparkContext são:

Configurar parâmetros internos do Spark: Ele carrega as configurações da sua aplicação, como o nome da aplicação, o modo de execução (local ou em cluster) e a quantidade de recursos a serem alocados.

Conectar-se ao gerenciador do cluster: Seja ele YARN, Mesos ou o gerenciador standalone do próprio Spark.

Criar RDDs (Resilient Distributed Datasets): RDDs são as estruturas de dados fundamentais do Spark, representando uma coleção de elementos particionada e imutável que pode ser operada em paralelo. 
O SparkContext é a porta de entrada para a criação e manipulação desses RDDs.

A importância do .getOrCreate()
O método .getOrCreate() possui um comportamento crucial:

Se já existir um SparkContext ativo na sua aplicação, ele simplesmente o retorna.

Se não houver um SparkContext ativo, ele cria um novo.

Esse comportamento é essencial porque, em uma mesma aplicação (mais especificamente, em uma mesma Java Virtual Machine - JVM), apenas um SparkContext pode estar ativo por vez. 
Tentar criar um novo SparkContext quando um já existe resultaria em um erro. O .getOrCreate() elegantemente resolve esse problema, garantindo que sua aplicação sempre utilize
uma única instância do SparkContext, seguindo um padrão de projeto conhecido como Singleton.


"""

# Inicializar o SparkContext
sc = SparkContext.getOrCreate()

In [97]:
"""

Cria e configura o ponto de entrada principal para utilizar as funcionalidades do Spark, especialmente para trabalhar com DataFrames e SQL.

A variável spark passa a ser o seu "portal" para interagir com o Spark

Essa linha utiliza um padrão de projeto chamado "Builder" (ou Construtor), onde você encadeia vários métodos para configurar um objeto antes de criá-lo. Vamos ver cada parte:

SparkSession
É a classe principal, o ponto de entrada unificado para uma aplicação Spark a partir da versão 2.0. Ela engloba funcionalidades que antes eram separadas em diferentes "contextos" (como o SparkContext e SQLContext).

.builder
Este é um método que inicia o processo de construção de uma SparkSession. Ele retorna um objeto "construtor" que permite que você encadeie as configurações desejadas. 
Pense nele como o início da "planta" da sua aplicação.

.appName('PySpark DataFrame From RDD')
Este método define um nome para a sua aplicação. O nome que você define aqui ('PySpark DataFrame From RDD') aparecerá na interface de usuário do Spark e nos logs. 
Isso é extremamente útil para monitorar e identificar sua aplicação quando várias estiverem rodando em um mesmo cluster.

.getOrCreate()
Este é o método final e crucial. Ele faz duas coisas:

Get (Obter): Verifica se já existe uma SparkSession ativa nesta aplicação. Se sim, ele simplesmente retorna essa sessão já existente.

Create (Criar): Se nenhuma SparkSession estiver ativa, ele cria uma nova com as configurações que você definiu (como o appName) e a retorna.

Esse comportamento garante que você não crie acidentalmente múltiplas sessões do Spark, o que causaria erros. É uma forma segura e inteligente de inicialização


"""

# Criar o ponto de entrada para utilizar as funcionalidades do Spark
spark = SparkSession.builder.appName('PySpark DataFrame From RDD').getOrCreate()

In [98]:
"""

Este comando é uma das formas mais diretas de se criar um RDD (Resilient Distributed Dataset) no Spark.
A sua principal função é pegar uma coleção de dados que já existe na memória do seu programa principal (o "driver") e transformá-la em um conjunto de dados distribuído que o Spark pode processar em paralelo.

sc
Como já vimos, é o seu SparkContext, o ponto de entrada para a manipulação de RDDs.

.parallelize(...)
Este é o método chave aqui. O nome "parallelize" (paralelizar) é bastante descritivo: ele pega uma coleção de dados comum do Python (neste caso, uma lista) e a distribui (paraleliza)
entre os diferentes nós de trabalho (workers) do seu cluster Spark. É a forma mais comum de criar RDDs para fins de teste, aprendizado ou para processar dados que já estão na memória.

[('C', ...), ('B', ...), ...]
Este é o primeiro argumento do método: uma lista ([]) de tuplas (()). Esta é a coleção de dados que você quer distribuir.
No momento, esta lista existe apenas na memória do nó "driver", onde seu script está sendo executado.

, 4
Este segundo argumento é extremamente importante. Ele especifica em quantas partições o RDD deve ser dividido.

"""

# Criar um RDD (Resilient Distributed Dataset)
rdd = sc.parallelize([('C',85,76,87,91), ('B',85,76,87,91), ("A", 85,78,96,92), ("A", 92,76,89,96)], 4)

In [99]:
# Verificar o tipo de dados do objeto
print(type(rdd))

<class 'pyspark.core.rdd.RDD'>


In [100]:
# Criar uma lista com os cabeçalhos ou schema da tabela 
sub = ['id_person','value_1','value_2','value_3','value_4']

In [101]:
# Criar o data frame
marks_df = spark.createDataFrame(rdd, schema=sub)

In [102]:
# Ver o tipo do dado do objeto 
print(type(marks_df))

<class 'pyspark.sql.classic.dataframe.DataFrame'>


In [103]:
# Ver os Schema dos dados
marks_df.printSchema()

root
 |-- id_person: string (nullable = true)
 |-- value_1: long (nullable = true)
 |-- value_2: long (nullable = true)
 |-- value_3: long (nullable = true)
 |-- value_4: long (nullable = true)



In [104]:
# Ver os dados do data frame 
marks_df.show()

+---------+-------+-------+-------+-------+
|id_person|value_1|value_2|value_3|value_4|
+---------+-------+-------+-------+-------+
|        C|     85|     76|     87|     91|
|        B|     85|     76|     87|     91|
|        A|     85|     78|     96|     92|
|        A|     92|     76|     89|     96|
+---------+-------+-------+-------+-------+



In [105]:
# Fazer a instancia do Spark
spark = SparkSession.builder.appName('pysparkdf').getOrCreate()

In [106]:
# Baixar o arquivo pois não conseguimos usar o Spark para ler diretamnte o arquivo no Github
url = "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/Framework-de-Big-Data/Aula%202/cereal.csv"
local_path = "cereal.csv"
urllib.request.urlretrieve(url, local_path)

# Ler com Spark
df = spark.read.csv(local_path, sep=",", header=True, inferSchema=True)

In [107]:
# Ver o schema da tabela 
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [108]:
# Selecionar as colunas 
df.select("name","mfr","rating").show()


+--------------------+---+---------+
|                name|mfr|   rating|
+--------------------+---+---------+
|           100% Bran|  N|68.402973|
|   100% Natural Bran|  Q|33.983679|
|            All-Bran|  K|59.425505|
|All-Bran with Ext...|  K|93.704912|
|      Almond Delight|  R|34.384843|
|Apple Cinnamon Ch...|  G|29.509541|
|         Apple Jacks|  K|33.174094|
|             Basic 4|  G|37.038562|
|           Bran Chex|  R|49.120253|
|         Bran Flakes|  P|53.313813|
|        Cap'n'Crunch|  Q|18.042851|
|            Cheerios|  G|50.764999|
|Cinnamon Toast Cr...|  G|19.823573|
|            Clusters|  G|40.400208|
|         Cocoa Puffs|  G|22.736446|
|           Corn Chex|  R|41.445019|
|         Corn Flakes|  K|45.863324|
|           Corn Pops|  K|35.782791|
|       Count Chocula|  G|22.396513|
|  Cracklin' Oat Bran|  K|40.448772|
+--------------------+---+---------+
only showing top 20 rows


In [109]:
# Outra alternativa para o select 
df.select(*["name","mfr","rating"]).show()

+--------------------+---+---------+
|                name|mfr|   rating|
+--------------------+---+---------+
|           100% Bran|  N|68.402973|
|   100% Natural Bran|  Q|33.983679|
|            All-Bran|  K|59.425505|
|All-Bran with Ext...|  K|93.704912|
|      Almond Delight|  R|34.384843|
|Apple Cinnamon Ch...|  G|29.509541|
|         Apple Jacks|  K|33.174094|
|             Basic 4|  G|37.038562|
|           Bran Chex|  R|49.120253|
|         Bran Flakes|  P|53.313813|
|        Cap'n'Crunch|  Q|18.042851|
|            Cheerios|  G|50.764999|
|Cinnamon Toast Cr...|  G|19.823573|
|            Clusters|  G|40.400208|
|         Cocoa Puffs|  G|22.736446|
|           Corn Chex|  R|41.445019|
|         Corn Flakes|  K|45.863324|
|           Corn Pops|  K|35.782791|
|       Count Chocula|  G|22.396513|
|  Cracklin' Oat Bran|  K|40.448772|
+--------------------+---+---------+
only showing top 20 rows


In [110]:
"""

O comando df.withColumn("Calories",df['calories'].cast("Integer")).printSchema() realiza duas operações encadeadas: primeiro, modifica o DataFrame e, em seguida, exibe a estrutura resultante.

Parte 1: df['calories'].cast("Integer")
Esta é a operação mais interna e define o conteúdo da nova coluna.

df['calories']: Aqui, você está selecionando a coluna que já existe no seu DataFrame df com o nome "calories" (provavelmente com 'c' minúsculo). O resultado é um objeto do tipo Column.

.cast("Integer"): Este método é aplicado à coluna "calories". A função .cast() é usada para converter o tipo de dado de uma coluna. 
Neste caso, ele tenta converter todos os valores da coluna "calories" para o tipo Integer (número inteiro). Isso é útil, 
por exemplo, se a coluna foi lida de um arquivo como texto (string), mas você sabe que ela representa valores numéricos.

Parte 2: df.withColumn("Calories", ...)
Esta é a principal função de transformação do DataFrame.

df.withColumn(...): Este método é usado para adicionar uma nova coluna a um DataFrame ou substituir uma coluna existente com o mesmo nome.
É importante notar que, por causa da imutabilidade do Spark, ele não altera o DataFrame df original; ele retorna um novo DataFrame com a modificação.

"Calories": Este é o primeiro argumento e define o nome da nova coluna. Note que aqui foi usado "Calories" com 'C' maiúsculo.

df['calories'].cast("Integer"): Este é o segundo argumento, que define os valores que a nova coluna terá. Como vimos na Parte 1, são os valores da coluna antiga "calories" convertidos para inteiro.

Comportamento importante:

Se já existir uma coluna chamada "Calories" (com 'C' maiúsculo) no df, ela será sobrescrita com os novos valores.

Se não existir, uma nova coluna chamada "Calories" será adicionada ao DataFrame.

Parte 3: .printSchema()
Esta é uma ação, um comando que instrui o Spark a executar as transformações e mostrar um resultado.

.printSchema(): Este método exibe a "planta" ou esquema (schema) do DataFrame. Ele mostra uma lista de todas as colunas,
seus respectivos tipos de dados (ex: string, integer, double) e se elas podem ou não conter valores nulos (nullable).

Juntando Tudo
O fluxo de execução é o seguinte:

O Spark pega o DataFrame original df.

Ele cria um novo DataFrame adicionando uma coluna chamada "Calories".

O conteúdo dessa nova coluna é obtido pegando a coluna existente "calories" e convertendo seus valores para o tipo Integer.

Finalmente, o método .printSchema() é chamado sobre este novo DataFrame modificado, imprimindo sua estrutura no console. Você verá a lista de colunas, e a coluna "Calories" aparecerá com o tipo integer.

"""

# Alterar o nome da coluna e o tipo de dados
df.withColumn("Calories",df['calories'].cast("Integer")).printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- Calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [111]:
#Group by dos dados por name
df.groupBy("name","calories").count().show()

+--------------------+--------+-----+
|                name|calories|count|
+--------------------+--------+-----+
|Just Right Fruit ...|     140|    1|
|         Raisin Bran|     120|    1|
|Shredded Wheat sp...|      90|    1|
|           Corn Pops|     110|    1|
|  Honey Nut Cheerios|     110|    1|
|Muesli Raisins; D...|     150|    1|
|      Fruity Pebbles|     110|    1|
|           100% Bran|      70|    1|
|       Fruitful Bran|     120|    1|
|         Puffed Rice|      50|    1|
|      Raisin Squares|      90|    1|
|   Total Raisin Bran|     140|    1|
|      Golden Grahams|     110|    1|
|   Nutri-grain Wheat|      90|    1|
|   100% Natural Bran|     120|    1|
|Apple Cinnamon Ch...|     110|    1|
|Mueslix Crispy Blend|     160|    1|
|Shredded Wheat 'n...|      90|    1|
|              Smacks|     110|    1|
|      Quaker Oatmeal|     100|    1|
+--------------------+--------+-----+
only showing top 20 rows


In [112]:
#Group by dos dados por calories
df.groupBy("calories").count().show()

+--------+-----+
|calories|count|
+--------+-----+
|     140|    3|
|     120|   10|
|     100|   17|
|     130|    2|
|      50|    3|
|      80|    1|
|     160|    1|
|      70|    2|
|      90|    7|
|     110|   29|
|     150|    2|
+--------+-----+



In [113]:
#Order by dos dados
df.orderBy("protein").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|        Cap'n'Crunch|  Q|   C|     120|      1|  2|   220|  0.0| 12.0|    12|    35|      25|    2|   1.0|0.75|18.042851|
|Cinnamon Toast Cr...|  G|   C|     120|      1|  3|   210|  0.0| 13.0|     9|    45|      25|    2|   1.0|0.75|19.823573|
|         Cocoa Puffs|  G|   C|     110|      1|  1|   180|  0.0| 12.0|    13|    55|      25|    2|   1.0| 1.0|22.736446|
|           Corn Pops|  K|   C|     110|      1|  0|    90|  1.0| 13.0|    12|    20|      25|    2|   1.0| 1.0|35.782791|
|       Count Chocula|  G|   C|     110|      1|  1|   180|  0.0| 12.0|    13|    65|      25|    2|   1.0| 1.0|22.396513|
|      Frosted F

In [114]:
#Order by dos dados
df.orderBy("calories").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|         Puffed Rice|  Q|   C|      50|      1|  0|     0|  0.0| 13.0|     0|    15|       0|    3|   0.5| 1.0|60.756112|
|        Puffed Wheat|  Q|   C|      50|      2|  0|     0|  1.0| 10.0|     0|    50|       0|    3|   0.5| 1.0|63.005645|
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|      Shredded 

In [115]:
#Usando o when (case when)
df.select("name","vitamins", when(df.vitamins >= "25","rich in vitamins")).show()

+--------------------+--------+----------------------------------------------------+
|                name|vitamins|CASE WHEN (vitamins >= 25) THEN rich in vitamins END|
+--------------------+--------+----------------------------------------------------+
|           100% Bran|      25|                                    rich in vitamins|
|   100% Natural Bran|       0|                                                NULL|
|            All-Bran|      25|                                    rich in vitamins|
|All-Bran with Ext...|      25|                                    rich in vitamins|
|      Almond Delight|      25|                                    rich in vitamins|
|Apple Cinnamon Ch...|      25|                                    rich in vitamins|
|         Apple Jacks|      25|                                    rich in vitamins|
|             Basic 4|      25|                                    rich in vitamins|
|           Bran Chex|      25|                                  

In [116]:
#Usando o when (case when) com alias
df.select(
    "name",
    "vitamins",
    when(df.vitamins >= 25, "rich in vitamins").alias("vitamins_category")
).show()

+--------------------+--------+-----------------+
|                name|vitamins|vitamins_category|
+--------------------+--------+-----------------+
|           100% Bran|      25| rich in vitamins|
|   100% Natural Bran|       0|             NULL|
|            All-Bran|      25| rich in vitamins|
|All-Bran with Ext...|      25| rich in vitamins|
|      Almond Delight|      25| rich in vitamins|
|Apple Cinnamon Ch...|      25| rich in vitamins|
|         Apple Jacks|      25| rich in vitamins|
|             Basic 4|      25| rich in vitamins|
|           Bran Chex|      25| rich in vitamins|
|         Bran Flakes|      25| rich in vitamins|
|        Cap'n'Crunch|      25| rich in vitamins|
|            Cheerios|      25| rich in vitamins|
|Cinnamon Toast Cr...|      25| rich in vitamins|
|            Clusters|      25| rich in vitamins|
|         Cocoa Puffs|      25| rich in vitamins|
|           Corn Chex|      25| rich in vitamins|
|         Corn Flakes|      25| rich in vitamins|


In [117]:
#Realizando operação de filtro
df.filter(df.calories =="100").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|         Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|Cream of Wheat (Q...|  N|   H|     100|      3|  0|    80|  1.0| 21.0|     0|    -1|       0|    2|   1.0| 1.0|64.533816|
|Crispy Wheat & Ra...|  G|   C|     100|      2|  1|   140|  2.0| 11.0|    10|   120|      25|    3|   1.0|0.75|36.176196|
|         Double Chex|  R|   C|     100|      2|  0|   190|  1.0| 18.0|     5|    80|      25|    3|   1.0|0.75|44.330856|
| Frosted Mini-Wheats|  K|   C|     100|      3|  0|     0|  3.0| 14.0|     7|   100|      25|    2|   1.0| 0.8|58.345141|
|        Golden 

In [118]:
#Realizando operação de filtro
df.filter(df.calories >="100").show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon Ch...|  G|   C|     110|      2|  2|   180|  1.5| 10.5|    10|    70|      25|    1|   1.0|0.75|29.509541|
|         Apple Jacks|  K|   C|     110|      2|  0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|
|             Basic 4|  G|   C|     130|      3|  2|   210|  2.0| 18.0|     8|   100|      25|    3|  1.33|0.75|37.038562|
|        Cap'n'C

In [119]:
#Filtro usando o is not null 
df.filter(df.name.isNotNull()).show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [120]:
#Filtro usando o is null 
df.filter(df.name.isNull()).show()

+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+
|name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|rating|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+



#### Aula 3 - Consultas e Seleções

In [121]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [122]:
# Validando o sql
df = spark.sql(''' select 'OK' as Status''')
df.show()

+------+
|Status|
+------+
|    OK|
+------+



In [123]:
# Baixar o arquivo pois não conseguimos usar o Spark para ler diretamnte o arquivo no Github
url = "https://raw.githubusercontent.com/FIAP/Pos_Tech_DTAT/refs/heads/Framework-de-Big-Data/Aula%202/cereal.csv"
local_path = "cereal.csv"
urllib.request.urlretrieve(url, local_path)

# Ler com Spark
df = spark.read.csv(local_path, sep=",", header=True, inferSchema=True)
df.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [124]:
# Criar uma tabela temporaria na sessão atual
df.createOrReplaceTempView("cereal")

In [125]:
# Realizar o select via sql
cereal = spark.sql(''' SELECT * FROM cereal ''')
cereal.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [126]:
# Diferença entre o df.show e cereal.show() é o que df.show é via Python e o cereal.show() é via SQL
# As duas formas utiliza o mesmo motor que é o Spark
df.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [127]:
# Realizar o select via sql com filtro
cereal = spark.sql(''' SELECT * FROM cereal WHERE TYPE = 'C' ''')
cereal.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [128]:
# Realizar o filtro via Python
df.where(df['type'] == 'C').show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [129]:
# Realizar o select via sql com filtro
cereal = spark.sql(''' SELECT * FROM cereal WHERE mfr = 'G' ''')
cereal.show()
print()
print(f"Qtd de dados: {cereal.count()}")

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|Apple Cinnamon Ch...|  G|   C|     110|      2|  2|   180|  1.5| 10.5|    10|    70|      25|    1|   1.0|0.75|29.509541|
|             Basic 4|  G|   C|     130|      3|  2|   210|  2.0| 18.0|     8|   100|      25|    3|  1.33|0.75|37.038562|
|            Cheerios|  G|   C|     110|      6|  2|   290|  2.0| 17.0|     1|   105|      25|    1|   1.0|1.25|50.764999|
|Cinnamon Toast Cr...|  G|   C|     120|      1|  3|   210|  0.0| 13.0|     9|    45|      25|    2|   1.0|0.75|19.823573|
|            Clusters|  G|   C|     110|      3|  2|   140|  2.0| 13.0|     7|   105|      25|    3|   1.0| 0.5|40.400208|
|         Cocoa 

In [130]:
# Realizar o filtro via Python
df.where(df['mfr'] == 'G').show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|Apple Cinnamon Ch...|  G|   C|     110|      2|  2|   180|  1.5| 10.5|    10|    70|      25|    1|   1.0|0.75|29.509541|
|             Basic 4|  G|   C|     130|      3|  2|   210|  2.0| 18.0|     8|   100|      25|    3|  1.33|0.75|37.038562|
|            Cheerios|  G|   C|     110|      6|  2|   290|  2.0| 17.0|     1|   105|      25|    1|   1.0|1.25|50.764999|
|Cinnamon Toast Cr...|  G|   C|     120|      1|  3|   210|  0.0| 13.0|     9|    45|      25|    2|   1.0|0.75|19.823573|
|            Clusters|  G|   C|     110|      3|  2|   140|  2.0| 13.0|     7|   105|      25|    3|   1.0| 0.5|40.400208|
|         Cocoa 

In [131]:
# Realizar o select + count via sql com filtro
cereal = spark.sql(''' SELECT COUNT(*) AS Qtd FROM cereal WHERE mfr = 'G' ''')
cereal.show()

+---+
|Qtd|
+---+
| 22|
+---+



In [132]:
# Realizar a contagem via python
df.where(df['mfr'] == 'G').count()

22

In [133]:
# Ver o Schema dos dados
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)



In [134]:
# Criar uma tabela temporaria na sessão atual
df.createOrReplaceTempView("cereal")

In [135]:
# Fazer o select dos dados
cereal = spark.sql(''' SELECT name, type, mfr FROM cereal''')
cereal.show()

+--------------------+----+---+
|                name|type|mfr|
+--------------------+----+---+
|           100% Bran|   C|  N|
|   100% Natural Bran|   C|  Q|
|            All-Bran|   C|  K|
|All-Bran with Ext...|   C|  K|
|      Almond Delight|   C|  R|
|Apple Cinnamon Ch...|   C|  G|
|         Apple Jacks|   C|  K|
|             Basic 4|   C|  G|
|           Bran Chex|   C|  R|
|         Bran Flakes|   C|  P|
|        Cap'n'Crunch|   C|  Q|
|            Cheerios|   C|  G|
|Cinnamon Toast Cr...|   C|  G|
|            Clusters|   C|  G|
|         Cocoa Puffs|   C|  G|
|           Corn Chex|   C|  R|
|         Corn Flakes|   C|  K|
|           Corn Pops|   C|  K|
|       Count Chocula|   C|  G|
|  Cracklin' Oat Bran|   C|  K|
+--------------------+----+---+
only showing top 20 rows


In [136]:
# Fazer o select com distinct dos dados
cereal = spark.sql(''' SELECT DISTINCT type, mfr FROM cereal ORDER BY type, mfr''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+----+---+
|type|mfr|
+----+---+
|   C|  G|
|   C|  K|
|   C|  N|
|   C|  P|
|   C|  Q|
|   C|  R|
|   H|  A|
|   H|  N|
|   H|  Q|
+----+---+

Qtd de dados: 9


In [137]:
# Fazer o select com where dos dados
cereal = spark.sql(''' SELECT * FROM cereal WHERE mfr = 'K' ''')
print(f"Qtd de dados: {cereal.count()}")

Qtd de dados: 23


In [138]:
# Fazer o select com where dos dados
cereal = spark.sql(''' SELECT * FROM cereal WHERE calories = 100 ''')
print(f"Qtd de dados: {cereal.count()}")

Qtd de dados: 17


In [139]:
# Fazer o select com where dos dados
cereal = spark.sql(''' SELECT * FROM cereal WHERE mfr = 'K' and calories = 100 ''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+-------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|               name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+-------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|        Corn Flakes|  K|   C|     100|      2|  0|   290|  1.0| 21.0|     2|    35|      25|    1|   1.0| 1.0|45.863324|
|Frosted Mini-Wheats|  K|   C|     100|      3|  0|     0|  3.0| 14.0|     7|   100|      25|    2|   1.0| 0.8|58.345141|
|         Product 19|  K|   C|     100|      3|  0|   320|  1.0| 20.0|     3|    45|     100|    3|   1.0| 1.0| 41.50354|
+-------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+

Qtd de dados: 3


In [140]:
# Fazer o group by 
cereal = spark.sql(''' SELECT mfr, type, COUNT(*) AS total, SUM(calories) as total_calories FROM cereal GROUP BY mfr, type ORDER BY mfr, type''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+---+----+-----+--------------+
|mfr|type|total|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  G|   C|   22|          2450|
|  K|   C|   23|          2500|
|  N|   C|    5|           420|
|  N|   H|    1|           100|
|  P|   C|    9|           980|
|  Q|   C|    7|           660|
|  Q|   H|    1|           100|
|  R|   C|    8|           920|
+---+----+-----+--------------+

Qtd de dados: 9


In [141]:
# Fazer o group by de forma identada
cereal = spark.sql(
    ''' 

    SELECT 
        mfr, type, COUNT(*) AS total, SUM(calories) as total_calories 
        
    FROM 
        cereal 
    
    GROUP BY 
        mfr, type 
        
    ORDER BY 
        mfr, type

    ''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+---+----+-----+--------------+
|mfr|type|total|total_calories|
+---+----+-----+--------------+
|  A|   H|    1|           100|
|  G|   C|   22|          2450|
|  K|   C|   23|          2500|
|  N|   C|    5|           420|
|  N|   H|    1|           100|
|  P|   C|    9|           980|
|  Q|   C|    7|           660|
|  Q|   H|    1|           100|
|  R|   C|    8|           920|
+---+----+-----+--------------+

Qtd de dados: 9


In [142]:
# Fazer o select distinct
cereal = spark.sql(
    ''' 

    SELECT DISTINCT
        type
        
    FROM 
        cereal 

    ''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+----+
|type|
+----+
|   C|
|   H|
+----+

Qtd de dados: 2


In [143]:
# Fazer o case when
cereal = spark.sql(
    ''' 

    SELECT 
        mfr, 
        
        type,
        case when type = 'C' then 'A' else 'B' end as type_ney,

        COUNT(*) AS total,
        SUM(calories) as total_calories 
        
    FROM 
        cereal 
    
    GROUP BY 
        mfr, type 

    ''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+---+----+--------+-----+--------------+
|mfr|type|type_ney|total|total_calories|
+---+----+--------+-----+--------------+
|  A|   H|       B|    1|           100|
|  P|   C|       A|    9|           980|
|  K|   C|       A|   23|          2500|
|  G|   C|       A|   22|          2450|
|  Q|   C|       A|    7|           660|
|  R|   C|       A|    8|           920|
|  Q|   H|       B|    1|           100|
|  N|   H|       B|    1|           100|
|  N|   C|       A|    5|           420|
+---+----+--------+-----+--------------+

Qtd de dados: 9


In [144]:
# Fazer o case when
cereal = spark.sql(
    ''' 

    SELECT 
        mfr, 
        
        type,

        case 
            when type = 'C' then 'A' 
            when type = 'H' then 'B' 
            else 'C' 
            
        end as type_ney,

        COUNT(*) AS total,
        SUM(calories) as total_calories 
        
    FROM 
        cereal 
    
    GROUP BY 
        mfr, type 

    ''')
cereal.show()
print(f"Qtd de dados: {cereal.count()}")

+---+----+--------+-----+--------------+
|mfr|type|type_ney|total|total_calories|
+---+----+--------+-----+--------------+
|  A|   H|       B|    1|           100|
|  P|   C|       A|    9|           980|
|  K|   C|       A|   23|          2500|
|  G|   C|       A|   22|          2450|
|  Q|   C|       A|    7|           660|
|  R|   C|       A|    8|           920|
|  Q|   H|       B|    1|           100|
|  N|   H|       B|    1|           100|
|  N|   C|       A|    5|           420|
+---+----+--------+-----+--------------+

Qtd de dados: 9


In [145]:
# Fazer select de todas as tabelas 
cereal = spark.sql (""" select * from cereal """)
cereal.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+
|           100% Bran|  N|   C|      70|      4|  1|   130| 10.0|  5.0|     6|   280|      25|    3|   1.0|0.33|68.402973|
|   100% Natural Bran|  Q|   C|     120|      3|  5|    15|  2.0|  8.0|     8|   135|       0|    3|   1.0| 1.0|33.983679|
|            All-Bran|  K|   C|      70|      4|  1|   260|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|
|Apple Cinnamon 

In [146]:
# Agrupar dados
cereal = spark.sql (""" 
                    
    SELECT
        mfr,
        type,
        sum(calories) as sum_calories,
        min(calories) as min_calories,
        max(calories) as max_calories,
        avg(calories) as avg_calories,
        count(distinct name) as count_distinct_names,
        count(name) as count_names
    
    FROM
        cereal
    
    GROUP BY
        mfr,
        type
    
    ORDER BY
            mfr,
            type

        """)
cereal.show()

+---+----+------------+------------+------------+------------------+--------------------+-----------+
|mfr|type|sum_calories|min_calories|max_calories|      avg_calories|count_distinct_names|count_names|
+---+----+------------+------------+------------+------------------+--------------------+-----------+
|  A|   H|         100|         100|         100|             100.0|                   1|          1|
|  G|   C|        2450|         100|         140|111.36363636363636|                  22|         22|
|  K|   C|        2500|          50|         160|108.69565217391305|                  23|         23|
|  N|   C|         420|          70|          90|              84.0|                   5|          5|
|  N|   H|         100|         100|         100|             100.0|                   1|          1|
|  P|   C|         980|          90|         120|108.88888888888889|                   9|          9|
|  Q|   C|         660|          50|         120| 94.28571428571429|              

In [147]:
# Agrupar dados
cereal = spark.sql (""" 
                    
    SELECT
        mfr,
        type,
                    
        sum(calories) as sum_calories,
        min(calories) as min_calories,
        max(calories) as max_calories,
        avg(calories) as avg_calories,

        sum(carbo) as sum_carbo,
        min(carbo) as min_carbo,
        max(carbo) as max_carbo,
        avg(carbo) as avg_carbo,

        sum(vitamins) as sum_vitamins,
        min(vitamins) as min_vitamins,
        max(vitamins) as max_vitamins,
        avg(vitamins) as avg_vitamins,        

        count(distinct name) as count_distinct_names,
        count(name) as count_names
    
    FROM
        cereal
    
    GROUP BY
        mfr,
        type
    
    ORDER BY
            mfr,
            type

        """)
cereal.show()

+---+----+------------+------------+------------+------------------+---------+---------+---------+------------------+------------+------------+------------+------------------+--------------------+-----------+
|mfr|type|sum_calories|min_calories|max_calories|      avg_calories|sum_carbo|min_carbo|max_carbo|         avg_carbo|sum_vitamins|min_vitamins|max_vitamins|      avg_vitamins|count_distinct_names|count_names|
+---+----+------------+------------+------------+------------------+---------+---------+---------+------------------+------------+------------+------------+------------------+--------------------+-----------+
|  A|   H|         100|         100|         100|             100.0|     16.0|     16.0|     16.0|              16.0|          25|          25|          25|              25.0|                   1|          1|
|  G|   C|        2450|         100|         140|111.36363636363636|    324.0|     10.5|     21.0|14.727272727272727|         775|          25|         100| 35.2272

In [148]:
# Agrupar dados
cereal = spark.sql (""" 
                    
    SELECT
        mfr,
        type,
                    
        sum(calories) as sum_calories,
        min(calories) as min_calories,
        max(calories) as max_calories,
        cast(avg(calories) as decimal(10,2)) as avg_calories,

        sum(carbo) as sum_carbo,
        min(carbo) as min_carbo,
        max(carbo) as max_carbo,
        cast(avg(carbo) as decimal(10,2)) as avg_carbo,

        sum(vitamins) as sum_vitamins,
        min(vitamins) as min_vitamins,
        max(vitamins) as max_vitamins,
        cast(avg(vitamins) as decimal(10,2)) as avg_vitamins,        

        count(distinct name) as count_distinct_names,
        count(name) as count_names
    
    FROM
        cereal
    
    GROUP BY
        mfr,
        type
    
    ORDER BY
            mfr,
            type

        """)
cereal.show()

+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|mfr|type|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count_distinct_names|count_names|
+---+----+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|  A|   H|         100|         100|         100|      100.00|     16.0|     16.0|     16.0|    16.00|          25|          25|          25|       25.00|                   1|          1|
|  G|   C|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|                  22|         22|
|  K|   C|        2500|          50|         160|      108.7

In [149]:
# Agrupar dados com case 
cereal = spark.sql (""" 
                    
    SELECT
        mfr,
        type,
                    
        case
            when mfr = 'A' then 'Abacaxi'       
            when mfr = 'G' then 'Goiaba'   
            when mfr = 'K' then 'Banana'   
            when mfr = 'N' then 'Maça'   
            when mfr = 'P' then 'Tomate'   
            when mfr = 'Q' then 'Pera'   
            when mfr = 'R' then 'Uva'   
            else 'NA' 
        end 
                    
        as type_fruit,
      
        sum(calories) as sum_calories,
        min(calories) as min_calories,
        max(calories) as max_calories,
        cast(avg(calories) as decimal(10,2)) as avg_calories,

        sum(carbo) as sum_carbo,
        min(carbo) as min_carbo,
        max(carbo) as max_carbo,
        cast(avg(carbo) as decimal(10,2)) as avg_carbo,

        sum(vitamins) as sum_vitamins,
        min(vitamins) as min_vitamins,
        max(vitamins) as max_vitamins,
        cast(avg(vitamins) as decimal(10,2)) as avg_vitamins,        

        count(distinct name) as count_distinct_names,
        count(name) as count_names
    
    FROM
        cereal
    
    GROUP BY
        mfr,
        type
    
    ORDER BY
            mfr,
            type

        """)
cereal.show()

+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|mfr|type|type_fruit|sum_calories|min_calories|max_calories|avg_calories|sum_carbo|min_carbo|max_carbo|avg_carbo|sum_vitamins|min_vitamins|max_vitamins|avg_vitamins|count_distinct_names|count_names|
+---+----+----------+------------+------------+------------+------------+---------+---------+---------+---------+------------+------------+------------+------------+--------------------+-----------+
|  A|   H|   Abacaxi|         100|         100|         100|      100.00|     16.0|     16.0|     16.0|    16.00|          25|          25|          25|       25.00|                   1|          1|
|  G|   C|    Goiaba|        2450|         100|         140|      111.36|    324.0|     10.5|     21.0|    14.73|         775|          25|         100|       35.23|                  22|         22|
|  K|

In [ ]:
# Importar base Sales
url = "https://github.com/FIAP/Pos_Tech_DTAT/raw/refs/heads/Framework-de-Big-Data/Aula%203/sales_data_sample.csv"
local_path = "sales_data_sample.csv"
urllib.request.urlretrieve(url, local_path)

sales = spark.read.csv(local_path, sep=",", header=True, inferSchema=True)

In [ ]:
# Ver os dados 
sales.show()

+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|      ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|           PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|   STATE|POSTALCODE|  COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|DEALSIZE|
+-----------+---------------+---------+---------------+-------+---------------+-------+------+--------+-------+-----------+----+-----------+--------------------+----------------+--------------------+------------+-------------+--------+----------+---------+---------+---------------+----------------+--------+
|      10107|             30|     95.7|              2| 2871.0| 2/24/2003

In [154]:
sales.printSchema()

root
 |-- ORDERNUMBER: integer (nullable = true)
 |-- QUANTITYORDERED: integer (nullable = true)
 |-- PRICEEACH: double (nullable = true)
 |-- ORDERLINENUMBER: integer (nullable = true)
 |-- SALES: double (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- QTR_ID: integer (nullable = true)
 |-- MONTH_ID: integer (nullable = true)
 |-- YEAR_ID: integer (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: integer (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- ADDRESSLINE1: string (nullable = true)
 |-- ADDRESSLINE2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = tr

In [ ]:
# Criar tabela 
sales.createOrReplaceTempView('sales')

In [176]:
# Realizar join

calendar = spark.sql( """ 

    SELECT DISTINCT
        orderdate,
        qtr_id,
        month_id,
        year_id
                     
    FROM
        sales
                     
    ORDER BY
        orderdate

""")

sales_data = spark.sql( """ 

    SELECT DISTINCT
        ordernumber,
        customername,
        sales,
        quantityordered,
        productcode,
        orderlinenumber,
        priceeach               
                                    
    FROM
        sales
                     
    ORDER BY
        ordernumber

""")

customers = spark.sql( """ 

    SELECT DISTINCT 
        customername,
        phone,
        addressline1,
        addressline2,
        city,
        state,
        postalcode,
        country,
        territory

    FROM 
        sales
                  
    ORDER BY 
        customername

""")

# Criar Temp View para cada variavel criada
sales_data.createOrReplaceTempView('sales_data')
calendar.createOrReplaceTempView('calendar')
customers.createOrReplaceTempView('customers')

In [177]:
# Count dos dados
print(f"calendar count: {calendar.count()}")
print(f"sales count: {sales.count()}")
print(f"customers count: {customers.count()}")

calendar count: 252
sales count: 2823
customers count: 92


In [181]:
# Realizar Inner Join
master = spark.sql (""" 


SELECT DISTINCT 
    s.ordernumber, c.city
                    
FROM 
    sales_data s

INNER JOIN customers c 
    ON s.customername=c.customername


""")

master.show()

+-----------+-------------+
|ordernumber|         city|
+-----------+-------------+
|      10300|    Frankfurt|
|      10385|   San Rafael|
|      10241|   Strasbourg|
|      10182|   San Rafael|
|      10140|   Burlingame|
|      10153|       Madrid|
|      10293|       Torino|
|      10161|      Aaarhus|
|      10406|    Kobenhavn|
|      10414|       Boston|
|      10311|       Madrid|
|      10357|   San Rafael|
|      10195| White Plains|
|      10422|    Allentown|
|      10189|     Pasadena|
|      10111|San Francisco|
|      10204|          NYC|
|      10304|   Versailles|
|      10151|         Oulu|
|      10290|   Brickhaven|
+-----------+-------------+
only showing top 20 rows


#### Aula 4 - Operações entre Dataframes e Armazenamento

#### Aula 5 - Introdução aos Sistemas de Recomendação

#### Aula 6 - Recomendações com o Algoritmo ALS